# Access and plot the 3D reconstructions of the wing motor neurons and potential song-circuit neurons
For both the neurons and the neuropils, I found the right link to the precomputed volumes via Neuroglancer (e.g. [B2 MN via neuPrint+](https://neuprint.janelia.org/results?dataset=manc%3Av1.0&qt=findneurons&q=1&qr%5B0%5D%5Bcode%5D=fn&qr%5B0%5D%5Bds%5D=manc%3Av1.0&qr%5B0%5D%5Bpm%5D%5Bdataset%5D=manc%3Av1.0&qr%5B0%5D%5Bpm%5D%5Ball_segments%5D=false&qr%5B0%5D%5Bpm%5D%5Benable_contains%5D=true&qr%5B0%5D%5Bpm%5D%5Bneuron_name%5D=b2%20MN&qr%5B0%5D%5BvisProps%5D%5BrowsPerPage%5D=25&qr%5B1%5D%5Bcode%5D=ng&qr%5B1%5D%5Bds%5D=manc%3Av1.0&qr%5B1%5D%5Bpm%5D%5Bdataset%5D=manc%3Av1.0&qr%5B1%5D%5Bpm%5D%5Bskip%5D=true&qr%5B1%5D%5Bpm%5D%5BbodyIds%5D=10064&qr%5B2%5D%5Bcode%5D=sk&qr%5B2%5D%5Bds%5D=manc%3Av1.0&qr%5B2%5D%5Bpm%5D%5Bdataset%5D=manc%3Av1.0&qr%5B2%5D%5Bpm%5D%5Bskip%5D=true&qr%5B2%5D%5Bpm%5D%5BbodyIds%5D%5B0%5D=10064&qr%5B2%5D%5Bpm%5D%5Bcoordinates%5D=33687.56576720625%2C29580.296815531947%2C4277.065579838272%2C33687.56576720625%2C29580.296815531947%2C42758.94923094767&tab=1&ftab=)). In the Neuroglancer window right-click on the box that says '2 manc:v1.0' in the top left corner. A panel opens on the right and in the 'Source' tab the link to the precomputed volumes is provided. 
It works the same for the neuropils. Just right-click on the box '5 neuropils' and the link is provided in the panel on the right. Under the 'Seg.' tab this panel also provides the IDs for the different neuropils, which you will need for automated downloading.

This notbook creates differnt 3D plots with the wing motor neurons and the potential song-circuit neurons as well as some neuropils. The 3D reconstructions are loaded from the hard-drive if possible or downloaded otherwise. 

Author: Bjarne Schultze <br>
last modified: 01.12.2023

In [1]:
import pandas as pd
import numpy as np
from matplotlib import colors
import plotly

# Re-create deprecated aliases which are necessary for the navis module
np.int = np.int_
np.bool = np.bool_
import navis.interfaces.neuprint as neu
import navis

import neuron_loading_3d as nload

c:\Users\schul\anaconda3\envs\rms_analysis\lib\site-packages\python_jsonschema_objects\__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")
INFO  : cloud-volume successfully patched! (navis)


##### Load the necessary data

In [6]:
# Load the neuron property table
neuron_props = pd.read_pickle("neuron_prop_table.pkl")
# Load the neuron ID dictionary
neuron_ID_dict = np.load('./neuron_ID_dict.npy', allow_pickle=True).item()

In [3]:
# Load the 3D neuron reconstructions
mn_meshes = nload.load_neurons(neuron_props['bodyId'], neuron_ID_dict)

# Load the 3D roi reconstructions
roi_ID_dict = {13:"IntTct",14:"LTct",17:"WTct (L)",18:"WTct (R)",19:"HTct (L)",20:"HTct (R)"}
roi_meshes = nload.load_rois(list(roi_ID_dict.keys()), roi_ID_dict)

Importing:   0%|          | 0/68 [00:00<?, ?it/s]

Importing:   0%|          | 0/6 [00:00<?, ?it/s]

##### 3D plot showing the neurons (potential song circuit neuorns highlighted)

In [5]:
# Define a set of colors
cmap_colors = ['firebrick','darkturquoise','limegreen','royalblue','hotpink','lightskyblue','gold','purple','slategray','tomato','yellowgreen','plum','gray',
                'deepskyblue','aqua','darkorchid','darkblue','olivedrab','wheat','lightcyan','seagreen','bisque']
# Link colors and cell types (ensures same colors for same neuron type)
color_dict = dict(zip(np.unique(mn_meshes.name),cmap_colors))
# Create a list of colors for the 3D plot
neuron_cmap = []
for cell in mn_meshes.name:
    if any([check in cell for check in ['dPR1','TN1a','vPR6','pIP10']]):
        neuron_cmap.append(colors.to_rgba(color_dict[cell]))
    else:
        neuron_cmap.append([0.4,0.7,0.9,0.2])

# Plot the neurons
fig3d = navis.plot3d(mn_meshes, color=neuron_cmap, width=1200, height=800, inline=False)
# Remove legend entries for motor neurons
check_list = ['dPR1','TN1a','vPR6','pIP10']
for neuron in fig3d['data']:
    if not any([check in neuron.name for check in check_list]):
        neuron['showlegend'] = False
    else:
        check_list.remove(neuron.name)

# Save the plot as a html file 
_ = plotly.offline.plot(fig3d, filename='3d_neurons.html')
# fig3d.show()      # Uncomment for inline rendering, could make notebook very slow

INFO  : Use the `.show()` method to plot the figure. (navis)


##### 3D plot all neurons in different colors and show the relevalt neuropils

In [38]:
# Define colors for the ROIs (with transparency)
alpha = 0.4
roi_colors = [(0.1,0.9,0.8,alpha), (0.3,0.2,0.9,alpha), (0.9,0.4,0.2,alpha), (0.9,0.8,0.2,alpha), (0.8,0.2,0.9,alpha), (0.3,0.7,0.1,alpha)]

# Link colors and cell types (ensures same colors for same neuron type)
color_dict = dict(zip(neuron_props["type"].unique(),cmap_colors))
# Create a list of colors for the 3D plot
neuron_cmap = [ colors.to_rgba(color_dict[cell]) for cell in mn_meshes.name ]

# Combine ROI and neuron colors
[roi_colors.append(i) for i in neuron_cmap]
# Plot the neurons and the neuropils (ROIs)
fig3d = navis.plot3d([roi_meshes,mn_meshes], color=roi_colors, width=1200, height=800, inline=False)

# Reduce legend entries to one per neuorn type
check_list = []
for neuron in fig3d['data']:
    if any([check in neuron.name for check in check_list]):
        neuron['showlegend'] = False
    else:
        check_list.append(neuron.name)

# Save the plot as a html file and show it inline nevertheless
_ = plotly.offline.plot(fig3d, filename='3d_motorneurons_neuropils.html')
# fig3d.show()      # Uncomment for inline rendering, could make notebook very slow

INFO  : Use the `.show()` method to plot the figure. (navis)


#### Same visualization using the neuprint interface
Gives the same result after scaling but has the disadvantage that the neuropils cannot be hidden via the interactive figure legend.

For this to work you need an individual neuprint+ token which can be found in the *Account* section on the [neurprint+ website](https://neuprint.janelia.org/) after loggin in. 

In [ ]:
# Store individual neuPrint token
neuprint_token = ""

# Only continue if token was provided
if len(neuprint_token) > 0:
    # Creat neuPrint clienthttps://neuprint.janelia.org/
    neuprint_client = neu.Client('', dataset='manc:v1.0', token=neuprint_token)

    # Fetch ROIs
    neuropils = [neu.fetch_roi("WTct(UTct-T2)(L)"),neu.fetch_roi("WTct(UTct-T2)(R)"),neu.fetch_roi("IntTct"),
                 neu.fetch_roi("LTct"),neu.fetch_roi("HTct(UTct-T3)(L)"),neu.fetch_roi("HTct(UTct-T3)(R)")]
    # Apply scaling factor to match the scale of the neurons
    neuropils = [ neuropils[i] * 8 for i in range(len(neuropils)) ]

    # Color the neuropils with the defined colors
    for i in range(len(neuropils)):
        neuropils[i].color = roi_colors[i]
else:
    print('Please provide a neuprint token to proceed!')

In [ ]:
# Create a 3D plot of the neurons and the neuropils
navis.plot3d([neuropils,mn_meshes], color=roi_colors[-len(mn_meshes):], width=1000, height=800)